<a href="https://colab.research.google.com/github/gflores1092/AnalyticsSX/blob/main/An%C3%A1lisis_espacial_sellers_HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pytz
import datetime as dt
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

import gspread as gs
import gspread_dataframe as gd

import folium
from folium import plugins
from branca.colormap import LinearColormap, linear
from IPython.display import display

import itertools

from google.cloud import bigquery
from google.colab import auth
from google.colab import syntax

In [ ]:
pip install h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00


In [ ]:
import h3

In [ ]:
auth.authenticate_user()

project_id='bi-fcom-drmb-local-pe-sbx'        # Project ID
client = bigquery.Client(project=project_id)  # Create BigQuery Client

job_config = bigquery.QueryJobConfig()        # Job Configuration
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)  # Limit number of rows returned for testing

In [ ]:
query = syntax.sql('''
   SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs`.analisis_espacial_sellers
''')

In [ ]:
query_job = client.query(query, job_config=job_config)  # Execute the query
query_job.result()                                      # Wait for the query to complete
data = query_job.to_dataframe()                         # Fetch the results into a Pandas DataFrame

### **Funciones**

In [ ]:
def parse_polygon(polygon_text):
    polygon_text = polygon_text.replace('POLYGON', '').strip('()')
    coordinates = [list(map(float, pair.split())) for pair in polygon_text.split(', ')]
    coordinates = [[lon, lat] for lat, lon in coordinates]
    return coordinates

#### **Análisis Dropoff**

In [ ]:
do_data = (
    data
    .loc[:, ['punto_dropoff','nombre_dropoff','geolocation_dropoff', 'dropoff_polygon']]
    .drop_duplicates()
    .assign(**{'punto_dropoff': lambda x: x['punto_dropoff'].astype(str)})
    .query("~punto_dropoff.str.contains('Dropoff')")
    .query("~punto_dropoff.str.contains('None')")
    .query("~punto_dropoff.str.contains('Pickit 2 - LA VICTORIA')")  # Temporal
    .query("~punto_dropoff.isnull()").reset_index().drop(['index'],axis=1)
)

In [ ]:
st_data = (
    data
    .query("warehouse_name=='Main warehouse'")
    .assign(**{'punto_dropoff': lambda x: x['punto_dropoff'].astype(str)})
    .query("~punto_dropoff.str.contains('Dropoff')")
    .query("~punto_dropoff.str.contains('None')")
    .query("~punto_dropoff.str.contains('Pickit 2 - LA VICTORIA')")  # Temporal
    .query("~punto_dropoff.isnull()")
    .loc[:, ['seller_id','seller_name','address', 'punto_dropoff', 'solicitud_dropoff', 'geolocation_hd', 'geodesic_dist_hd_dropoff', 'manhattan_dist_hd_dropoff']]
    .drop_duplicates()
    .query("~geolocation_hd.isnull()").reset_index().drop(['index'],axis=1)
)

In [ ]:
# Set the height of the map container
map_height = '1200px'

# Create a map object
m = folium.Map(location=[-12.043501, -77.043921], zoom_start=12, control_scale=True, tiles=None).add_to(folium.Figure(height=map_height))

# Add a tile layer (you can choose your preferred tile source)
folium.TileLayer('openstreetmap', name='Base Map').add_to(m)

color_palette = [
  'purple', 'blue', 'darkblue', 'beige', 'gray', 'darkpurple', 'black', 'red', 'darkred', 'green', 'darkgreen', 'orange', 'lightblue', 'lightgreen', 'lightgray', 'lightred', 'pink', 'cadetblue', 'white'
]
color_cycle = itertools.cycle(color_palette)

# Group based on dropoff name

dropoff_name_colors = {}
dropoff_name_groups = {}

## 1. LAYERS

dropoff_layer = folium.FeatureGroup(name='Dropoff points')
seller_layer = folium.FeatureGroup(name='Seller points', show=False)

polygon_layer = folium.FeatureGroup(name='Dropoff Polygons')  # Set show=False to initially hide the layer

## 2. DATA

for index, row in do_data.iterrows():

    ## 1. Locations

    dropoff_location = row['geolocation_dropoff'].replace('POINT(', '').replace(')', '').split(' ')
    dropoff_latitude = float(dropoff_location[1])
    dropoff_longitude = float(dropoff_location[0])
    punto_dropoff = row['punto_dropoff']

    popup_content = f"<div style='width: 200px;'><strong>Punto:</strong> {row['punto_dropoff']}<br><strong>Nombre:</strong> {row['nombre_dropoff']}"

    if punto_dropoff not in dropoff_name_colors:
        dropoff_name_colors[punto_dropoff] = next(color_cycle)

    marker = folium.Marker([dropoff_latitude, dropoff_longitude], tooltip=popup_content, icon=folium.Icon(icon='home', color=dropoff_name_colors[punto_dropoff])).add_to(dropoff_layer)

    ## 2. Polygons

    dropoff_polygon = row['dropoff_polygon']
    polygon_coordinates = parse_polygon(dropoff_polygon)

    polygon = folium.Polygon(locations=polygon_coordinates, color=dropoff_name_colors[punto_dropoff], fill=True, fill_color=dropoff_name_colors[punto_dropoff], fill_opacity=0.2).add_to(polygon_layer)

    polygon_popup = folium.Popup(f"<div style='width: 200px;'><strong>Punto:</strong> {row['punto_dropoff']}<br><strong>Nombre:</strong> {row['nombre_dropoff']}")

    polygon.add_child(polygon_popup)


for index, row in st_data.iterrows():

    #### 2. Sellers

    seller_location = row['geolocation_hd'].replace('POINT(', '').replace(')', '').split(' ')
    seller_latitude = float(seller_location[1])
    seller_longitude = float(seller_location[0])
    punto_dropoff = row['punto_dropoff']

    seller_popup = f"<div style='width: 200px;'> \
                      <strong>Seller ID:</strong> {row['seller_id']} <br> \
                      <strong>Seller:</strong> {row['seller_name']} <br> \
                      <strong>Dropoff:</strong> {row['punto_dropoff']} <br> \
                      <strong>Solicitud:</strong> {row['solicitud_dropoff']} <br> \
                      <strong>Distancia Geodésica:</strong> {row['geodesic_dist_hd_dropoff']} <br> \
                      <strong>Distancia Manhattan:</strong> {row['manhattan_dist_hd_dropoff']}"

    folium.Marker([seller_latitude, seller_longitude], tooltip=seller_popup, icon=folium.Icon(icon='circle', color=dropoff_name_colors[punto_dropoff], opacity=0.25)).add_to(seller_layer)

## 3. MAP

# Add the store layer to the map
dropoff_layer.add_to(m)
seller_layer.add_to(m)

polygon_layer.add_to(m)

# Add the feature groups to the map
#for group_name, group in dropoff_name_groups.items():
#    group.add_to(m)

# Add layer control with custom labels
layer_control = folium.LayerControl()
#for group_name, group in dropoff_name_groups.items():
#    layer_control.add_child(folium.CustomHtml(f'<span style="margin-left: 10px;">{group_name}</span>'), group=group)

layer_control.add_to(m)

# Save the map as an HTML file
display(m)

In [ ]:
x = data \
  .query("warehouse_name=='Main warehouse'")

x['shipment_provider'] = x['shipment_provider'].fillna('')

x = x[x['shipment_provider'].str.lower().str.contains('pickit|dropoff')] \
  .loc[:,
         [
            'ruc', 'seller_id', 'seller_name', 'shipment_provider',
            'contact_name', 'phone', 'email', 'country', 'region',
            'city', 'district', 'address', 'postcode', 'periferico', 'big_ticket',
            'diff_address', 'start_time', 'close_time', 'work_days', 'monday',
            'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
            'latitude', 'longitude', 'macronodo', 'motivo', 'status_sabado',
            'punto_dropoff', 'direccion_dropoff', 'nombre_dropoff',
            'horario_dropoff', 'frecuencia_dropoff', 'solicitud_dropoff', 'PM', 'CT', 'division',
            'category', 'location_fsc', 'location_hd', 'phone_fsc', 'phone_hd',
            'district_fsc', 'district_hd', 'address_fsc', 'address_hd',
            'start_time_fsc', 'start_time_hd',
            'geodesic_dist_hd_dropoff',
            'manhattan_dist_hd_dropoff'
         ]] \
  .rename(columns={'geodesic_dist_hd_dropoff': 'geodesic_distance', 'manhattan_dist_hd_dropoff': 'manhattan_distance'})

<ipython-input-16-97ef4161e3b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['shipment_provider'] = x['shipment_provider'].fillna('')


In [ ]:
x.to_excel('filtered_data.xlsx', index=False)

Export KMLs

In [ ]:
pip install simplekml

In [ ]:
import simplekml

In [ ]:
import colorsys

# Define the number of colors you want
num_colors = 30

# Create an empty list to store the colors
colors = []

# Generate equally spaced hues
hue_values = [i / num_colors for i in range(num_colors)]

# Generate distinct colors with varying hues
for hue in hue_values:
    h = hue  # Hue
    s = 0.9  # Saturation
    l = 0.6  # Lightness
    rgb = colorsys.hls_to_rgb(h, l, s)
    r, g, b = [int(x * 255) for x in rgb]
    color_hex = f'#{r:02X}{g:02X}{b:02X}'
    colors.append(color_hex)

# Print the list of distinct colors
print(colors)

In [ ]:
dropoff_color_map = {}

1. Dropoff points

In [ ]:
# Create a KML object for Dropoff Points
dropoff_kml = simplekml.Kml()

# Iterate through your dropoff data
for index, row in do_data.iterrows():
    # Extract seller location
    dropoff_location = row['geolocation_dropoff'].replace('POINT(', '').replace(')', '').split(' ')
    dropoff_latitude = float(dropoff_location[1])
    dropoff_longitude = float(dropoff_location[0])

    # Get the dropoff name
    dropoff_name = row['punto_dropoff']

    # Assign a unique color based on the dropoff name or retrieve the color if it already exists
    if dropoff_name in dropoff_color_map:
        dropoff_color = dropoff_color_map[dropoff_name]
    else:
        # Use a color from the list and remove it so it won't be reused
        dropoff_color = colors.pop(0)
        dropoff_color_map[dropoff_name] = dropoff_color

    # Create a Placemark with the custom style
    placemark = dropoff_kml.newpoint(name=row['punto_dropoff'], coords=[(dropoff_longitude, dropoff_latitude)])
    placemark.style.iconstyle.icon.href = 'https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png'
    placemark.style.iconstyle.color = dropoff_color

# Save the Dropoff Points KML file
dropoff_kml.save("dropoff_points.kml")

2. Dropoff polygons

In [ ]:
def parse_polygon_(polygon_text):
    polygon_text = polygon_text.replace('POLYGON', '').strip('()')
    coordinates = [list(map(float, pair.split())) for pair in polygon_text.split(', ')]
    # Reverse the order of coordinates to (lon, lat)
    coordinates = [[lat, lon] for lat, lon in coordinates]
    return coordinates

In [ ]:
# Create a KML object for Dropoff Polygons
dropoff_polygon_kml = simplekml.Kml()

# Iterate through your dropoff data
for index, row in do_data.iterrows():
    # Extract dropoff polygon coordinates
    dropoff_polygon = row['dropoff_polygon']
    polygon_coordinates = parse_polygon_(dropoff_polygon)

    # Get or assign a color for the current dropoff name
    dropoff_name = row['punto_dropoff']

    # Assign a unique color based on the dropoff name or retrieve the color if it already exists
    if dropoff_name in dropoff_color_map:
        dropoff_color = dropoff_color_map[dropoff_name]
    else:
        # Use a color from the list and remove it so it won't be reused
        dropoff_color = colors.pop(0)
        dropoff_color_map[dropoff_name] = dropoff_color

    # Create a Placemark with the custom style
    placemark = dropoff_polygon_kml.newpolygon(name=row['punto_dropoff'], outerboundaryis=polygon_coordinates)
    placemark.style.polystyle.color = dropoff_color
    placemark.style.polystyle.opacity = 0.5

# Save the Dropoff Polygons KML file
dropoff_polygon_kml.save("dropoff_polygons.kml")

3. Seller points

In [ ]:
# Create a KML object for Seller Points
seller_kml = simplekml.Kml()

# Iterate through your seller data
for index, row in st_data.iterrows():
    # Extract seller location
    seller_location = row['geolocation_hd'].replace('POINT(', '').replace(')', '').split(' ')
    seller_latitude = float(seller_location[1])
    seller_longitude = float(seller_location[0])

    # Get the dropoff name
    dropoff_name = row['punto_dropoff']

    # Assign a unique color based on the dropoff name or retrieve the color if it already exists
    if dropoff_name in dropoff_color_map:
        dropoff_color = dropoff_color_map[dropoff_name]
    else:
        # Use a color from the list and remove it so it won't be reused
        dropoff_color = colors.pop(0)
        dropoff_color_map[dropoff_name] = dropoff_color

    # Create a Placemark with the custom style
    placemark = seller_kml.newpoint(name=row['seller_name'], coords=[(seller_longitude, seller_latitude)])
    placemark.style.iconstyle.icon.href = 'https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png'
    placemark.style.iconstyle.color = dropoff_color

# Save the Seller Points KML file
seller_kml.save("seller_points.kml")

#### **Análisis Hexágonos**

In [ ]:
hexagons = data \
            .query("warehouse_name=='Main warehouse'") \
            .groupby(['h3_8_geo_hd'], dropna=False) \
            .agg(sellers=('seller_id', 'nunique')) \
            .sort_values(['sellers'], ascending=False) \
            .reset_index()

In [ ]:
# Create a map centered at a specific location
m = folium.Map(location=[-12.043501, -77.043921], zoom_start=12)

# Palette
colors = ['#0000FF', '#008000', '#FFFF00', '#FFA500', '#FF0000', '#800080', '#FF00FF', '#00FFFF', '#FF5733', '#3333FF']  # 10 Custom colors
values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# Create a colormap dictionary with color ranges
colormap = {}
for i in range(len(values) - 1):
    colormap[values[i]] = colors[i]

# Add a hexbin layer with colors based on the 'sellers' column
for index, row in hexagons.iterrows():
    hexagon = h3.h3_to_geo_boundary(row['h3_8_geo_hd'])
    sellers = row['sellers']

    color = None
    for value, clr in colormap.items():
        if sellers < value:
            color = clr
            break
    if color is None:
        color = colors[-1]

    popup = f"<div style='width: 200px;'> \
              <strong>Hexagon ID:</strong> {row['h3_8_geo_hd']} <br> \
              <strong>Sellers:</strong> {row['sellers']}"

    folium.Polygon(
        locations=hexagon,
        fill=True,
        color=color,
        fill_opacity=0.6,
        popup=folium.Popup(popup)
    ).add_to(m)

display(m)

In [ ]:
data[(data['h3_8_geo_hd']=='888e629997fffff') & (data['warehouse_name']=='Main warehouse')][['seller_id','seller_name','shipment_provider']]